In [ ]:
import pymunk
import pygame
import pymunk.pygame_util

In [ ]:
pygame.init()
screen = pygame.display.set_mode((600, 600))
draw_options = pymunk.pygame_util.DrawOptions(screen)
clock = pygame.time.Clock()
running = True

space = pymunk.Space()
space.gravity = 0, 100

a1  = pymunk.Body(body_type=pymunk.Body.STATIC)
a1.position = 100,500
c1 = pymunk.Circle(a1, radius=5)
c1.friction = 0.7 
c1.elasticity = 0.1
a2  = pymunk.Body(body_type=pymunk.Body.STATIC)
a2.position = 500,500
c2 = pymunk.Circle(a2, radius=5)
c2.friction = 0.7
c2.elasticity = 0.1
space.add(a1,c1,a2,c2)

In [ ]:
space.iterations = 300
space.damping = 0.3
roads = []
joints = []  # Store joints to access their forces later

for x in range(0, 4):
    r = pymunk.Body(mass=5, moment=5)
    r.position = 150 + x * 100, 500
    print(r.position)
    rod = pymunk.Segment(r, (-51, 0), (51, 0), 10)
    rod.friction = 0.8
    rod.elasticity = 0.1  
    
    if(x == 0):
        joint = pymunk.constraints.PivotJoint(a1, r, (0, 0), (-51, 0))
    else:
        joint = pymunk.constraints.PivotJoint(roads[-1], r, (51, 0), (-51, 0))

    joint.collide_bodies = False
    roads.append(r)
    joints.append(joint)  # Store joint reference
    space.add(r, rod, joint)


final_joint = pymunk.constraints.PivotJoint(roads[-1], a2, (51, 0), (0, 0))
final_joint.collide_bodies = False
joints.append(final_joint)  
space.add(final_joint)


dt = 0.001  
while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

    screen.fill("purple")
    space.debug_draw(draw_options)
    
    space.step(dt)
    
    # Get tension for each joint
    for i, joint in enumerate(joints):
        tension = joint.impulse / dt
        print(f"Segment {i} tension: {tension:.2f}N")
    
    pygame.display.update()

pygame.quit()
